## Setup

In [1]:
import boto3
import json

def get_secret(secret_name, region_name="us-east-1"):
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name)
    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    get_secret_value_response = json.loads(get_secret_value_response['SecretString'])
    return get_secret_value_response

creds = get_secret("wysde")
USERNAME = creds["RDS_MYSQL_USERNAME"]
PASSWORD = creds["RDS_MYSQL_PASSWORD"]
HOST = creds["RDS_MYSQL_HOST"]
DATABASE = 'sparsh'

conn_str = 'mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(USERNAME, PASSWORD, HOST, DATABASE)

%config SqlMagic.autopandas=True
%config SqlMagic.displaycon=False
%config SqlMagic.feedback=False
%config SqlMagic.displaylimit=5
%reload_ext sql
%sql {conn_str}

Let’s assume we have a table named “**employees**” with the following columns:

- **id:** the unique identifier for each employee
- **name**: the name of the employee
- **gender:** the gender of the employee
- **salary:** the salary of the employee
- **department:** the department of the employee

Here’s the MySQL script to create the `employees` table and insert the sample data:

In [3]:
%%sql

DROP TABLE IF EXISTS employees;

CREATE TABLE employees (
  id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(50) NOT NULL,
  gender VARCHAR(10) NOT NULL,
  salary INT NOT NULL,
  department VARCHAR(50) NOT NULL
);


INSERT INTO employees (name, gender, salary, department) VALUES
('Ramesh Gupta', 'Male', 55000, 'Sales'),
('Priya Sharma', 'Female', 65000, 'Marketing'),
('Sanjay Singh', 'Male', 75000, 'Sales'),
('Anjali Verma', 'Female', 45000, 'Finance'),
('Rajesh Sharma', 'Male', 80000, 'Marketing'),
('Smita Patel', 'Female', 60000, 'HR'),
('Vikram Yadav', 'Male', 90000, 'Sales'),
('Neha Sharma', 'Female', 55000, 'Marketing'),
('Rahul Singh', 'Male', 70000, 'Finance'),
('Sonali Gupta', 'Female', 50000, 'Sales');

SELECT * FROM employees;

,id,name,gender,salary,department
0,1,Ramesh Gupta,Male,55000,Sales
1,2,Priya Sharma,Female,65000,Marketing
2,3,Sanjay Singh,Male,75000,Sales
3,4,Anjali Verma,Female,45000,Finance
4,5,Rajesh Sharma,Male,80000,Marketing
5,6,Smita Patel,Female,60000,HR
6,7,Vikram Yadav,Male,90000,Sales
7,8,Neha Sharma,Female,55000,Marketing
8,9,Rahul Singh,Male,70000,Finance
9,10,Sonali Gupta,Female,50000,Sales


## Queries

### Write a query to find the average salary of male and female employees in each department.

In [4]:
%%sql

SELECT department, gender, AVG(salary) AS avg_salary
FROM employees
GROUP BY department, gender;

,department,gender,avg_salary
0,Sales,Male,73333.3333
1,Marketing,Female,60000.0000
2,Finance,Female,45000.0000
3,Marketing,Male,80000.0000
4,HR,Female,60000.0000
5,Finance,Male,70000.0000
6,Sales,Female,50000.0000


### Write a query to find the name and salary of the employee with the highest salary in each department.

In [5]:
%%sql

SELECT department, name, salary
FROM employees
WHERE (department, salary) IN (
  SELECT department, MAX(salary)
  FROM employees
  GROUP BY department
);

,department,name,salary
0,Marketing,Rajesh Sharma,80000
1,HR,Smita Patel,60000
2,Sales,Vikram Yadav,90000
3,Finance,Rahul Singh,70000


### Write a query to find the names of employees who earn more than the average salary in their department.

In [6]:
%%sql

SELECT name, salary, department
FROM employees
WHERE salary > (
  SELECT AVG(salary)
  FROM employees AS e2
  WHERE e2.department = employees.Department
);

,name,salary,department
0,Sanjay Singh,75000,Sales
1,Rajesh Sharma,80000,Marketing
2,Vikram Yadav,90000,Sales
3,Rahul Singh,70000,Finance


### Find the top 3 highest paid employees in each department.

In [7]:
%%sql

SELECT e.department,
    e.name,
    e.salary
FROM employees e
WHERE (
        SELECT COUNT(*)
        FROM employees
        WHERE department = e.department
            AND salary > e.salary
    ) < 3;

,department,name,salary
0,Sales,Ramesh Gupta,55000
1,Marketing,Priya Sharma,65000
2,Sales,Sanjay Singh,75000
3,Finance,Anjali Verma,45000
4,Marketing,Rajesh Sharma,80000
5,HR,Smita Patel,60000
6,Sales,Vikram Yadav,90000
7,Marketing,Neha Sharma,55000
8,Finance,Rahul Singh,70000


### Find the names of employees who have a salary greater than the average salary of their department.

In [8]:
%%sql

SELECT e.name
FROM employees e
    JOIN (
        SELECT department,
            AVG(salary) AS avg_salary
        FROM employees
        GROUP BY department
    ) AS dept_avg ON e.department = dept_avg.department
WHERE e.salary > dept_avg.avg_salary;

,name
0,Sanjay Singh
1,Rajesh Sharma
2,Vikram Yadav
3,Rahul Singh


### Find the department(s) with the highest paid employee(s).

In [9]:
%%sql

WITH max_salary AS (
    SELECT department,
        MAX(salary) AS highest_salary
    FROM employees
    GROUP BY department
)
SELECT m.department,
    e.name,
    e.salary
FROM employees e
    JOIN max_salary m ON e.department = m.department
    AND e.salary = m.highest_salary;

,department,name,salary
0,Marketing,Rajesh Sharma,80000
1,HR,Smita Patel,60000
2,Sales,Vikram Yadav,90000
3,Finance,Rahul Singh,70000
